### Refs

- [RDKit Documentation](https://www.rdkit.org/docs/GettingStartedInPython.html)
- [PubChem](https://pubchem.ncbi.nlm.nih.gov/) - find SDF files for molecules
- [YouTube Tutorial](https://www.youtube.com/watch?v=NozaWUkJ3YM)


## Imports

In [7]:
from src.loader import SDFLoader
from src.core.atom import MBAtom, RELEVANT_OX_STATE_SYMBOLS, RELEVANT_RING_ATOMS
from rdkit.Chem import rdMolDescriptors as rdmd
from rdkit.Chem import Mol
from rdkit import Chem

### RDkit magnetic experiments
`Preliminary coding & studying RDKit`


In [ ]:
for i, mol in enumerate(SDFLoader.load(file="As-benzene-arsenic acid.sdf")):
    print(f"I am molecule {i + 1}")

    mol.preprocess()

    for atom in mol.GetAtoms():
        print(atom)

I am molecule 1
Atom: C,
  IsInRing: True,
  in_ring: True,
  GetNoBondedAtoms: 3,
  FormalCharge: 0,
  self_charge: None,
  ox_state: None,
  id: 0,
  neighbors: ['C', 'As', 'As']

Atom: C,
  IsInRing: True,
  in_ring: True,
  GetNoBondedAtoms: 3,
  FormalCharge: 0,
  self_charge: None,
  ox_state: None,
  id: 1,
  neighbors: ['C', 'C']

Atom: As,
  IsInRing: True,
  in_ring: True,
  GetNoBondedAtoms: 2,
  FormalCharge: 0,
  self_charge: None,
  ox_state: 3,
  id: 2,
  neighbors: ['C', 'C']

Atom: C,
  IsInRing: True,
  in_ring: True,
  GetNoBondedAtoms: 3,
  FormalCharge: 0,
  self_charge: None,
  ox_state: None,
  id: 3,
  neighbors: ['C', 'C']

Atom: C,
  IsInRing: True,
  in_ring: True,
  GetNoBondedAtoms: 3,
  FormalCharge: 0,
  self_charge: None,
  ox_state: None,
  id: 4,
  neighbors: ['As', 'C']

Atom: C,
  IsInRing: True,
  in_ring: True,
  GetNoBondedAtoms: 3,
  FormalCharge: 0,
  self_charge: None,
  ox_state: None,
  id: 5,
  neighbors: ['C', 'C']

Atom: As,
  IsInRing: Fa

In [9]:
from constants import PASCAL_CONST

sum_dia_contr = 0

for i in range(len(mols)):
    mol = mols[i]

    print(f"I am molecule {i + 1}")

    rdmd.CalcOxidationNumbers(mol)

    mol = Chem.AddHs(mol)
    atoms: list[Chem.Atom] = [
        a for a in mol.GetAtoms()
    ]  # Creates a list of atoms for each molecule

    for atom in atoms:
        symbol = atom.GetSymbol()
        if symbol in PASCAL_CONST:
            # retrieve const data
            covalent_data = PASCAL_CONST[symbol]["covalent"]
            ionic_data = PASCAL_CONST[symbol]["ionic"]["charge"]
            ox_state_data = PASCAL_CONST[symbol]["covalent"]["ox_state"]

            if in_ring:
                sum_dia_contr += covalent_data["ring"]

            """TODO: Resolve the error"""
            if not in_ring:
                sum_dia_contr += covalent_data["open_chain"]

            """TODO: Fix incompatible format of oxidation state. Ox_state from RDKit: int, Ox_state in PASCAL_CONST: string."""
            # if ox_state is not None:
            # ....

            """TODO: Fix incompatible format of charge. Charge from RDKit: int; Charge in PASCAL_CONST: string."""
            # if atom_charge is not None:
            # ....
    print(sum_dia_contr)

NameError: name 'mols' is not defined